## Importação das Bibiliotecas

In [1]:
import os


In [2]:
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-17\jdk-17.0.8"
os.environ["SPARK_HOME"] = r"C:\Users\leoba\Documents\Enem\Spark\spark-3.4.1-bin-hadoop3"
os.environ['HADOOP_HOME'] = r"C:\Users\leoba\Documents\Enem\Spark\spark-3.4.1-bin-hadoop3\hadoop"
os.environ["PYTHONPATH"] = os.path.join(os.environ["SPARK_HOME"], "python/lib/py4j-0.10.7-src.zip")

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()


In [5]:
import pyspark.sql.functions as F


# Testando configuração arquivo Parquet


In [6]:
file_csv = r'C:\Users\leoba\Documents\Enem\Dataset\Model\MICRODADOS_ENEM_MODEL.CSV'

#Função Paa ler os arquivos CSV
def lerCSV(spark, caminho_arquivo):
    df = spark.read.csv(caminho_arquivo, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')
    colunas_selecionadas = [F.col(df.columns[i]) for i in range(0, 13)] + [F.col(df.columns[i]) for i in range(23, 27)]  + [F.col(df.columns[i]) for i in range(31, 35)] + [F.col(df.columns[39])] + [F.col(df.columns[56])]
    return df.select(colunas_selecionadas)



In [7]:
dfDadosEnem = lerCSV(spark, file_csv)
dfDadosProva = lerCSV(spark, file_csv)

In [8]:
dfDadosEnem.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|CO_MUNICIPIO_ESC|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|TP_LINGUA|Q006|
+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+----+
|210057943671|  2022|             14|      M|              2|          2|               1|              1|              2|  

In [9]:
# Lista das colunas que não devem ser afetadas
colunas_nao_afetadas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT']


In [10]:
for coluna in dfDadosEnem.columns:
    if coluna not in colunas_nao_afetadas:
        dfDadosEnem = dfDadosEnem.withColumn(
            coluna,
            F.when(F.col(coluna).isNull(), -2).otherwise(F.col(coluna))
        )

In [11]:
dfPreencher = dfDadosEnem


# Cria arquivo parquet, através do DF Carregado

In [17]:
#dfPreencher.write.parquet(path=r"C:\Users\leoba\Documents\Enem\Dataset\parquet",mode='overwrite') Em teste

## Ler arquivo Avro recem criado

In [45]:
reader = DataFileReader(open("DadosEnem.avro", "rb"), DatumReader())
for NU_ANO in reader:
    print(NU_ANO)
reader.close()

InvalidAvroBinaryEncoding: Read 0 bytes, expected 4 bytes

## Criando uma SparkSession

## Processo de Leitura do CSV de Participantes


In [12]:
# Parte fixa do caminho
parte_fixa = r'Documents\Enem\Dataset\Model\MICRODADOS_ENEM_MODEL.CSV'

# Diretório raiz do sistema operacional atual
root_dir = os.path.expanduser("~")
print(root_dir)
# Caminho completo
caminho_model = os.path.join(root_dir, parte_fixa)
print(caminho_model)

C:\Users\leoba
C:\Users\leoba\Documents\Enem\Dataset\Model\MICRODADOS_ENEM_MODEL.CSV


In [13]:
#Função Paa ler os arquivos CSV
def lerCSV(spark, caminho_arquivo):
    df = spark.read.csv(caminho_model, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')
    colunas_selecionadas = df.columns[:12]
    return df.select(colunas_selecionadas)


In [14]:
##Chama função quer ler o arquivo CSV e Transforma para DF.SPARK

dfDadosModel = lerCSV(spark, caminho_model)



In [8]:
dfDadosModel.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|
+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+
|210057943671|  2022|             14|      M|              2|          2|               1|              1|              2|        1|     null|           0|
|210057516120|  2022|             14|      M|              2|          1|               1|              1|             16|        1|     null|           0|
|210055097896|  2022|              4|      M|              0|          3|               1|              1|              1|        1|     null|           0|
|210055724397|  2022|              6|      M|              1|   

In [15]:
PathParticipantes = r'.\Enem\Dataset\parquet'


In [16]:
# Parte fixa do caminho, Não alterar esse path
parte_fixa = r'Documents\Enem\Dataset\parquet'

# Caminho completo
caminho_model = os.path.join(root_dir, parte_fixa)
print(caminho_model)

# Salve o DataFrame em formato Parquet usando apenas o caminho como argumento
dfDadosModel.write.format("avro").save(caminho_model)


C:\Users\leoba\Documents\Enem\Dataset\parquet


AnalysisException: Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of Apache Avro Data Source Guide.

In [ ]:
            converted = convert_exception(e.java_exception)


In [ ]:
def install_exception_handler() -> None:


In [ ]:
class AnalysisException(CapturedException, BaseAnalysisException):
